# Implementing manual SGD on Linear regression model :



Here we will implement a function to manually perform <b> Stochastic Gradient Descent(SGD) </b> Optimization for Linear regression algorithm.

We will use boston house prices dataset from Sklearn for this exercise.

In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import euclidean
from sklearn.metrics import r2_score

### Loading data into DF and high level analysis:

In [2]:
b=datasets.load_boston()
boston_x=np.array(b.data)
boston_y=np.array(b.target)
boston_x1=np.nan_to_num(boston_x)

In [3]:
data=pd.DataFrame(boston_x)

In [4]:
data.columns=['crim','zn','indus','chas','nox','rm','age','dis','rad','tax','ptratio','black','istat']
data['Price']=boston_y
data.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,istat,Price
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [5]:
data.isnull().any()

crim       False
zn         False
indus      False
chas       False
nox        False
rm         False
age        False
dis        False
rad        False
tax        False
ptratio    False
black      False
istat      False
Price      False
dtype: bool

In [8]:
data.isna().any()

crim       False
zn         False
indus      False
chas       False
nox        False
rm         False
age        False
dis        False
rad        False
tax        False
ptratio    False
black      False
istat      False
Price      False
dtype: bool

In [9]:
data.describe()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,istat,Price
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.593761,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.596783,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.647423,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


### Splitting of data into Test and Train:

In [5]:
train,test=train_test_split(data,test_size=0.3,random_state=33)
train.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,istat,Price
198,0.03768,80.0,1.52,0.0,0.404,7.274,38.3,7.3090,2.0,329.0,12.6,392.20,6.62,34.6
317,0.24522,0.0,9.90,0.0,0.544,5.782,71.7,4.0317,4.0,304.0,18.4,396.90,15.94,19.8
356,8.98296,0.0,18.10,1.0,0.770,6.212,97.4,2.1222,24.0,666.0,20.2,377.73,17.60,17.8
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0
136,0.32264,0.0,21.89,0.0,0.624,5.942,93.5,1.9669,4.0,437.0,21.2,378.25,16.90,17.4


In [6]:
x_train=np.array(train.drop(columns='Price'))
y_train=np.array(train['Price'])
x_test=np.array(test.drop(columns='Price'))
y_test=np.array(test['Price'])

In [17]:
print(len(x_train))
print(len(x_test))

354
152


### Standardisation of data:

In [7]:
scaler=StandardScaler()

In [8]:
std_train=scaler.fit_transform(x_train)
std_test=scaler.transform(x_test)

### Functions used : 

I created multiple functions here to enable modularity in code, which promotes code reusability. 

In [9]:
def W_gradient(w,b,x,y):
    '''
    Gradient function with respect to W
    '''
    w_dash=(-2/len(x))*((x.T @ (y-(x @ (w.T))-b)))
    #print(w_dash.shape)
    return w_dash

In [10]:
def B_gradient(w,b,x,y):
    '''
    Gradient function with respect to B
    '''
    b_dash=((-2/len(x))*np.sum((y-(x.dot(w.T))-b)))
    #print(b_dash.shape)
    return b_dash
    

Ignore the below one as i was testing it and is not used here..

In [11]:
def update_function1(w,b,r,x,y,iteration,batch_size,tolerance=0.0001,max_iteration=100000):
    '''
    To perform update function for SGD Optimization and store optimum value of W and B to the variables
    '''
    global optimal_w
    global optimal_b
    
    for i in range(max_iteration):
        iteration+=1
        k=np.random.randint(0,high=len(x),size=batch_size) # Batch size
        x1=x[k]
        y1=y[k,np.newaxis]
        w_new= w.T -(r * W_gradient(w,b,x1,y1))
        w_new=w_new.T
        b_new=b-(r * B_gradient(w,b,x1,y1))
        
        if(euclidean(w_new,w) <= tolerance and euclidean(b_new,b) <= tolerance):
            print(f'\n Converged after {iteration} Iterations.. :) ')
            optimal_w= w.T
            optimal_b= b
            break
    
        elif(iteration==max_iteration):
            print(f'\n Convergence not happened within {max_iteration} iterations')
            break 
    
        else:    
            w=w_new
            b=b_new
            r*=0.999
            
    

In [12]:
def gradient_function(feats,class1,size=100):
    '''
    to calculate SGD for linear regression with batch = size
    '''
    global iteration
    learning_rate =10**-2
    weight=np.random.rand(1,(feats.shape[1]))
    b=np.random.rand()
    #print(weight.shape)
    #print(b.shape)
    iteration=0
    update_function1(weight,b,learning_rate,feats,class1,iteration,size)
    

In [13]:
def sgd_performance(y,y_hat):
    '''
    To Compute r^2 error and Mean squared error
    '''
    print(f'\n R2 Score is {r2_score(y,y_hat)}')
    print(f'\n Mean Squared error is = {mean_squared_error(y_test,y_hat)}')
    

In [14]:
def predictor(x_test,y_test):
    '''
    To predict and measure the performance of test data with Optimal W obtained from Gradient function
    '''
    pred=(x_test.dot(optimal_w))+optimal_b
    sgd_performance(y_test,pred)
    

### Analysis of Implemented program:

In [15]:
gradient_function(std_train,y_train,100)


 Converged after 4929 Iterations.. :) 


In [16]:
print(f'Optimal w is \n {optimal_w} \n\n and Optimal_b is {optimal_b}')

Optimal w is 
 [[-1.00381125]
 [ 1.05528851]
 [-0.19997677]
 [ 0.84941216]
 [-1.53512172]
 [ 2.9944787 ]
 [-0.14821994]
 [-3.08029167]
 [ 2.11409331]
 [-1.58632589]
 [-1.98326815]
 [ 0.52641895]
 [-3.87854776]] 

 and Optimal_b is 22.93860002860945


In [17]:
predictor(std_test,y_test)


 R2 Score is 0.6858200021185472

 Mean Squared error is = 22.70542604657475


### Linear regression Sklearn package:

In [18]:
lr=LinearRegression()
lr.fit(std_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [19]:
pred1=lr.predict(std_test)

In [20]:
print(mean_squared_error(y_test,pred1))

22.76048343651732


In [21]:
sgd_performance(y_test,pred1)


 R2 Score is 0.6850581608467723

 Mean Squared error is = 22.76048343651732


### Comparision between Sklearn and my implementation:

 By comparing the results from cell [20] and cell [24]. It is clear that my implementation of <i>SGD with <b>batch size 100 and tolerance as 0.0001 </b> is almost similar/slightly better to the results obtained from Sklearn package</i>.

### Analysis of our implemented SGD with different batch sizes:

In [23]:
def sgd_analyser():
    '''
    To analyse result of SGD with various batch sizes.
    '''
    batch_list=[10,50,100,175,250]
    for i in batch_list:
        print('\n ################################################################################')
        print(f'\n Batch Size = {i}')
        gradient_function(std_train,y_train,i)
        predictor(std_test,y_test)
    print('\n ################################################################################')

In [24]:
sgd_analyser()


 ################################################################################

 Batch Size = 10

 Converged after 4929 Iterations.. :) 

 R2 Score is 0.6853018337438443

 Mean Squared error is = 22.74287347731771

 ################################################################################

 Batch Size = 50

 Converged after 5296 Iterations.. :) 

 R2 Score is 0.6847504832849436

 Mean Squared error is = 22.78271893901078

 ################################################################################

 Batch Size = 100

 Converged after 5194 Iterations.. :) 

 R2 Score is 0.6854899340481515

 Mean Squared error is = 22.72927968529535

 ################################################################################

 Batch Size = 175

 Converged after 4352 Iterations.. :) 

 R2 Score is 0.6857509480126176

 Mean Squared error is = 22.710416507157753

 ################################################################################

 Batch Size = 250

 Converged after 4470 

### Inferences from above exercise:

From the above analysis we can infer the below points,

1.) With increase in batch size, number of iterations taken to converge decreases.

2.) There is not much variation in performance scores between different batch sizes as it is clear that Stochastic gradient can produce faster convergence with very small batch size rather than the conventional GD where entire set of data will be considered.

3.) When learning rate(r) is decreased on a faster rate in each iteration, it is producing minimum iteration for convergence, but the optimal_W is less accurate than the one obtained with the very slower decay.

4.)Since we are not using any regulariser in this linear regression exercise, we don't need any cross validation data sets/any hyperparameter tuning part.